# 3dasm Project

## Agenda
1. [Phase 1: Gaussian Process on Simulated Data only](#Phase-1:-Gaussian-Process-on-Simulated-Data-only)
2. [Training with Different Kernels: common Constant*RBF kernel with an optimizer fmin_1_bfgs_b](#Training-with-Different-Kernels:-common-Constant*RBF-kernel-with-an-optimizer-fmin_1_bfgs_b)
3. [Training with Different Kernels: ExpSineSquared with an optimizer fmin_1_bfgs_b](#Training-with-Different-Kernels:-ExpSineSquared-with-an-optimizer-fmin_1_bfgs_b)
4. [Training with Different Kernels: Matern with an optimizer fmin_1_bfgs_b](#Training-with-Different-Kernels:-Matern-with-an-optimizer-fmin_1_bfgs_b)
5. [Training with Different Kernels: common Constant*RBF kernel With No Optimizer](#Training-with-Different-Kernels:-common-Constant*RBF-kernel-With-No-Optimizer)
6. [Training with Different Kernels: ExpSineSquared with no optimizer](#Training-with-Different-Kernels:-ExpSineSquared-with-no-optimizer)
7. [Training with Different Kernels: Matern with no optimizer](#Training-with-Different-Kernels:-Matern-with-no-optimizer)
8. [Phase 1.5: Gaussian Process on Experimental Data Only](#Phase-1.5:-Gaussian-Process-on-Experimental-Data-Only)
9. [Training with Different kernels: common Constant*RBF kernel with an optimizer fmin_1_bfgs_b](#Training-with-Different-kernels:-common-Constant*RBF-kernel-with-an-optimizer-fmin_1_bfgs_b)
10. [Training with Different Kernels: ExpSineSquared with an optimizer fmin_1_bfgs_b](#Training-with-Different-Kernels:-ExpSineSquared-with-an-optimizer-fmin_1_bfgs_b)
11. [Training with Different Kernels: Matern with an optimizer fmin_1_bfgs_b](#Training-with-Different-Kernels:-Matern-with-an-optimizer-fmin_1_bfgs_b)
12. [Phase 2: Training with Simulation Data and Testing with Experimental Data](#Phase-2:-Training-with-Simulation-Data-and-Testing-with-Experimental-Data)
13. [Constant Kernel and  f_min optimizer](#Constant-Kernel-and-f_min-optimizer)
14. [Matern kernel with no optimizer](#Matern-kernel-with-no-optimizer)
15. [Phase 3: Multi-fidelity Gaussian Process](#Phase-3:-Multi-fidelity-Gaussian-Process)
16. [Phase 4: Error Metrics](#Phase-4:-Error-Metrics)

### Importing necessary packages

In [5]:
import matplotlib.pyplot as plt # import plotting tools to create figures
import numpy as np # import numpy to handle a lot of things!
from IPython.display import display, Math # to print with Latex math
import pandas as pd

%config InlineBackend.figure_format = "retina" # render higher resolution images in the notebook
plt.rcParams["figure.figsize"] = (8,4) # rescale figure size appropriately for slides

In [6]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, ExpSineSquared, ConstantKernel, WhiteKernel

<a id='Phase 1: Gaussian Process on Simulated Data only'></a>

## Phase 1: Gaussian Process on Simulated Data only

In [7]:
sim_df = pd.read_csv("../3dasm_data/neoHookean/data_0.00021544_0.001.csv") # read pandas DataFrame
print("Show the simulated DataFrameused for training data:\n", sim_df)

FileNotFoundError: [Errno 2] No such file or directory: 'bdata/neoHookean/data_0.00021544_0.001.csv'

In [ ]:
Data_x = sim_df.iloc[:,0] # select the input VALUES from your dataframe into Data_x
Data_y = sim_df.iloc[:,1] # select the output VALUES from your dataframe inta Data_y
print("Data_x is:\n",Data_x)
print("\nData_y is:\n",Data_y)

In [ ]:
fig_sim_data, ax_sim_data = plt.subplots() # create a plot
ax_sim_data.plot(Data_x, Data_y, 'b.')
ax_sim_data.set_xlabel("t (s)", fontsize=20) # create x-axis label with font size 20
ax_sim_data.set_ylabel("r (m)", fontsize=20) # create y-axis label with font size 20
ax_sim_data.set_title("Simulated Bubble Radius vs Time", fontsize=20); # create title with font size 20
plt.show(fig_sim_data)

In [ ]:
from sklearn.model_selection import train_test_split

n_data = 50 # number of points in our dataset
testset_ratio = 0.10 # ratio of test set points from the dataset

X_data = np.reshape(Data_x,(-1,1)) # a 2D array that scikit-learn likes

seed = 1987 # set a random seed so that everyone gets the same result
np.random.seed(seed)

# Let's split into 10% training points and the rest for testing:
X_train, X_test, y_train, y_test = train_test_split(X_data,
                                    Data_y, test_size=testset_ratio,
                                    random_state=seed)

x_train = X_train.ravel() # just for plotting later
x_test = X_test.ravel() # just for plotting later

print("Here's a print of X_train:\n", X_train)

In [ ]:
n_max = np.max(X_data)
print(n_max)

# Define points used for plotting
n_plot = 500 # number of points used for plotting the model
n_max = np.max(X_data)
x_plot = np.linspace(0, n_max, n_plot) # create dataset with uniformly spaced points
X_plot = np.reshape(x_plot,(-1,1)) # a 2D array that scikit-learn likes

<b id='Training with Different Kernels: common Constant*RBF kernel with an optimizer fmin_1_bfgs_b'></b>

### Training with Different Kernels: common Constant*RBF kernel with an optimizer fmin_1_bfgs_b

In [ ]:
# Define the kernel function
kernel = ConstantKernel(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2)) # This is the very common Constant*RBF kernel

gp_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, optimizer='fmin_l_bfgs_b', n_restarts_optimizer=20)
#gp_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-10, optimizer=None, n_restarts_optimizer=20)

# Fit to data to determine parameters
gp_model.fit(X_train, y_train)

# Make the prediction on the entire dataset (for plotting)
y_plot_pred, sigma_plot = gp_model.predict(X_plot, return_std=True) # also output the uncertainty (std)

# Predict for test set (for error metric)
y_pred, sigma = gp_model.predict(X_test, return_std=True) # also output the uncertainty (std)

In [ ]:
# Plot the prediction and the 95% confidence interval
fig1, ax1 = plt.subplots()

ax1.plot(x_plot, y_plot_pred, 'b-', label="GPR prediction")
ax1.fill(np.concatenate([x_plot, x_plot[::-1]]),
         np.concatenate([y_plot_pred - 1.9600 * sigma_plot,
                        (y_plot_pred + 1.9600 * sigma_plot)[::-1]]),
         alpha=.5, fc='b', ec='None', label='95% confidence interval')

#ax1.plot(x_train, y_train, 'ro', markersize=6, label="training points") # noiseless data
#ax1.plot(x_test, y_test, 'kX', markersize=6, label="testing points") # Plot test points

ax1.set_xlabel('$t (s)$', fontsize=20)
ax1.set_ylabel('$r (m)$', fontsize=20)
ax1.set_title("Posterior kernel: %s"
              % gp_model.kernel_, fontsize=20) # Show in the title the value of the hyperparameters
#ax1.set_ylim(-10, 15) # just to provide more space for the legend
ax1.legend(loc='upper left', fontsize=15)
fig1.set_size_inches(8,8)
plt.close(fig1) # close the plot to see it in next cell

In [ ]:
fig1

In [ ]:
# Plot the function, the prediction and the 95% confidence interval
fig2, ax2 = plt.subplots()

ax2.plot(x_plot, y_plot_pred, 'b-', label="GPR prediction")
ax2.fill(np.concatenate([x_plot, x_plot[::-1]]),
         np.concatenate([y_plot_pred - 1.9600 * sigma_plot,
                        (y_plot_pred + 1.9600 * sigma_plot)[::-1]]),
         alpha=.5, fc='b', ec='None', label='95% confidence interval')

ax2.plot(x_train, y_train, 'ro', markersize=6, label="training points") # noiseless data
ax2.plot(x_test, y_test, 'kX', markersize=6, label="testing points") # Plot test points

ax2.set_xlabel('$t (s)$', fontsize=20)
ax2.set_ylabel('$r (m)$', fontsize=20)
ax2.set_title("Posterior kernel: %s"
              % gp_model.kernel_, fontsize=20) # Show in the title the value of the hyperparameters
#ax2.set_ylim(-10, 15) # just to provide more space for the legend
ax2.legend(loc='upper left', fontsize=15)
fig2.set_size_inches(8,8)
plt.close(fig2) # close the plot to see it in next cell

In [ ]:
fig2

<c id='Training with Different Kernels: ExpSineSquared with an optimizer fmin_1_bfgs_b'></c>

### Training with Different Kernels: ExpSineSquared with an optimizer fmin_1_bfgs_b

In [ ]:
# Define the kernel function
kernel = ExpSineSquared(length_scale=3.0, periodicity=3.14,
                        length_scale_bounds=(0.1, 10.0),
                        periodicity_bounds=(0.1, 10)) * RBF(3.0, (1e-2, 1e2))

gp_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, optimizer='fmin_l_bfgs_b', n_restarts_optimizer=20)
#gp_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-10, optimizer=None, n_restarts_optimizer=20)

# Fit to data to determine parameters
gp_model.fit(X_train, y_train)

# Make the prediction on the entire dataset (for plotting)
y_plot_pred, sigma_plot = gp_model.predict(X_plot, return_std=True) # also output the uncertainty (std)

# Predict for test set (for error metric)
y_pred, sigma = gp_model.predict(X_test, return_std=True) # also output the uncertainty (std)

In [ ]:
# Plot the prediction and the 95% confidence interval
fig1, ax1 = plt.subplots()

ax1.plot(x_plot, y_plot_pred, 'b-', label="GPR prediction")
ax1.fill(np.concatenate([x_plot, x_plot[::-1]]),
         np.concatenate([y_plot_pred - 1.9600 * sigma_plot,
                        (y_plot_pred + 1.9600 * sigma_plot)[::-1]]),
         alpha=.5, fc='b', ec='None', label='95% confidence interval')

#ax1.plot(x_train, y_train, 'ro', markersize=6, label="training points") # noiseless data
#ax1.plot(x_test, y_test, 'kX', markersize=6, label="testing points") # Plot test points

ax1.set_xlabel('$t (s)$', fontsize=20)
ax1.set_ylabel('$r (m)$', fontsize=20)
ax1.set_title("Posterior kernel: %s"
              % gp_model.kernel_, fontsize=20) # Show in the title the value of the hyperparameters
#ax1.set_ylim(-10, 15) # just to provide more space for the legend
ax1.legend(loc='upper left', fontsize=15)
fig1.set_size_inches(8,8)
plt.close(fig1) # close the plot to see it in next cell

In [ ]:
fig1

In [ ]:
# Plot the function, the prediction and the 95% confidence interval
fig2, ax2 = plt.subplots()

ax2.plot(x_plot, y_plot_pred, 'b-', label="GPR prediction")
ax2.fill(np.concatenate([x_plot, x_plot[::-1]]),
         np.concatenate([y_plot_pred - 1.9600 * sigma_plot,
                        (y_plot_pred + 1.9600 * sigma_plot)[::-1]]),
         alpha=.5, fc='b', ec='None', label='95% confidence interval')

ax2.plot(x_train, y_train, 'ro', markersize=6, label="training points") # noiseless data
ax2.plot(x_test, y_test, 'kX', markersize=6, label="testing points") # Plot test points

ax2.set_xlabel('$x$', fontsize=20)
ax2.set_ylabel('$f(x)$', fontsize=20)
ax2.set_title("Posterior kernel: %s"
              % gp_model.kernel_, fontsize=20) # Show in the title the value of the hyperparameters
#ax2.set_ylim(-10, 15) # just to provide more space for the legend
ax2.legend(loc='upper left', fontsize=15)
fig2.set_size_inches(8,8)
plt.close(fig2) # close the plot to see it in next cell

In [ ]:
fig2

<d id='Training with Different Kernels: Matern with an optimizer fmin_1_bfgs_b'></d>

### Training with Different Kernels: Matern with an optimizer fmin_1_bfgs_b

In [ ]:
# Define the kernel function
kernel = Matern(length_scale=1.0, length_scale_bounds=(1e-2, 1e2),nu=1.5)

gp_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, optimizer='fmin_l_bfgs_b', n_restarts_optimizer=20)
#gp_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-10, optimizer=None, n_restarts_optimizer=20)

# Fit to data to determine parameters
gp_model.fit(X_train, y_train)

# Make the prediction on the entire dataset (for plotting)
y_plot_pred, sigma_plot = gp_model.predict(X_plot, return_std=True) # also output the uncertainty (std)

# Predict for test set (for error metric)
y_pred, sigma = gp_model.predict(X_test, return_std=True) # also output the uncertainty (std)

In [ ]:
# Plot the prediction and the 95% confidence interval
fig1, ax1 = plt.subplots()

ax1.plot(x_plot, y_plot_pred, 'b-', label="GPR prediction")
ax1.fill(np.concatenate([x_plot, x_plot[::-1]]),
         np.concatenate([y_plot_pred - 1.9600 * sigma_plot,
                        (y_plot_pred + 1.9600 * sigma_plot)[::-1]]),
         alpha=.5, fc='b', ec='None', label='95% confidence interval')

#ax1.plot(x_train, y_train, 'ro', markersize=6, label="training points") # noiseless data
#ax1.plot(x_test, y_test, 'kX', markersize=6, label="testing points") # Plot test points

ax1.set_xlabel('$t (s)$', fontsize=20)
ax1.set_ylabel('$r (m)$', fontsize=20)
ax1.set_title("Posterior kernel: %s"
              % gp_model.kernel_, fontsize=20) # Show in the title the value of the hyperparameters
#ax1.set_ylim(-10, 15) # just to provide more space for the legend
ax1.legend(loc='upper left', fontsize=15)
fig1.set_size_inches(8,8)
plt.close(fig1) # close the plot to see it in next cell

In [ ]:
fig1

In [ ]:
# Plot the function, prediction and the 95% confidence interval
fig2, ax2 = plt.subplots()

ax2.plot(x_plot, y_plot_pred, 'b-', label="GPR prediction")
ax2.fill(np.concatenate([x_plot, x_plot[::-1]]),
         np.concatenate([y_plot_pred - 1.9600 * sigma_plot,
                        (y_plot_pred + 1.9600 * sigma_plot)[::-1]]),
         alpha=.5, fc='b', ec='None', label='95% confidence interval')

ax2.plot(x_train, y_train, 'ro', markersize=6, label="training points") # noiseless data
ax2.plot(x_test, y_test, 'kX', markersize=6, label="testing points") # Plot test points

ax2.set_xlabel('$t (s)$', fontsize=20)
ax2.set_ylabel('$r (m)$', fontsize=20)
ax2.set_title("Posterior kernel: %s"
              % gp_model.kernel_, fontsize=20) # Show in the title the value of the hyperparameters
#ax2.set_ylim(-10, 15) # just to provide more space for the legend
ax2.legend(loc='upper left', fontsize=15)
fig2.set_size_inches(8,8)
plt.close(fig2) # close the plot to see it in next cell

In [ ]:
fig2

<e id='Training with Different Kernels: common Constant*RBF kernel With No Optimizer'></e>

### Training with Different Kernels: common Constant*RBF kernel With No Optimizer

In [ ]:
# Define the kernel function
kernel = ConstantKernel(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2)) # This is the very common Constant*RBF kernel

gp_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, optimizer=None, n_restarts_optimizer=20)

# Fit to data to determine parameters
gp_model.fit(X_train, y_train)

# Make the prediction on the entire dataset (for plotting)
y_plot_pred, sigma_plot = gp_model.predict(X_plot, return_std=True) # also output the uncertainty (std)

# Predict for test set (for error metric)
y_pred, sigma = gp_model.predict(X_test, return_std=True) # also output the uncertainty (std)

In [ ]:
# Plot the prediction and the 95% confidence interval
fig1, ax1 = plt.subplots()

ax1.plot(x_plot, y_plot_pred, 'b-', label="GPR prediction")
ax1.fill(np.concatenate([x_plot, x_plot[::-1]]),
         np.concatenate([y_plot_pred - 1.9600 * sigma_plot,
                        (y_plot_pred + 1.9600 * sigma_plot)[::-1]]),
         alpha=.5, fc='b', ec='None', label='95% confidence interval')

#ax1.plot(x_train, y_train, 'ro', markersize=6, label="training points") # noiseless data
#ax1.plot(x_test, y_test, 'kX', markersize=6, label="testing points") # Plot test points

ax1.set_xlabel('$t (s)$', fontsize=20)
ax1.set_ylabel('$r (m)$', fontsize=20)
ax1.set_title("Posterior kernel: %s"
              % gp_model.kernel_, fontsize=20) # Show in the title the value of the hyperparameters
#ax1.set_ylim(-10, 15) # just to provide more space for the legend
ax1.legend(loc='upper left', fontsize=15)
fig1.set_size_inches(8,8)
plt.close(fig1) # close the plot to see it in next cell

In [ ]:
fig1

<f id='Training with Different Kernels: ExpSineSquared with no optimizer'></f>

### Training with Different Kernels: ExpSineSquared with no optimizer

In [ ]:
# Define the kernel function
kernel = ExpSineSquared(length_scale=3.0, periodicity=3.14,
                        length_scale_bounds=(0.1, 10.0),
                        periodicity_bounds=(0.1, 10)) * RBF(3.0, (1e-2, 1e2))

gp_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, optimizer=None, n_restarts_optimizer=20)

# Fit to data to determine parameters
gp_model.fit(X_train, y_train)

# Make the prediction on the entire dataset (for plotting)
y_plot_pred, sigma_plot = gp_model.predict(X_plot, return_std=True) # also output the uncertainty (std)

# Predict for test set (for error metric)
y_pred, sigma = gp_model.predict(X_test, return_std=True) # also output the uncertainty (std)

In [ ]:
# Plot the prediction and the 95% confidence interval
fig1, ax1 = plt.subplots()

ax1.plot(x_plot, y_plot_pred, 'b-', label="GPR prediction")
ax1.fill(np.concatenate([x_plot, x_plot[::-1]]),
         np.concatenate([y_plot_pred - 1.9600 * sigma_plot,
                        (y_plot_pred + 1.9600 * sigma_plot)[::-1]]),
         alpha=.5, fc='b', ec='None', label='95% confidence interval')

#ax1.plot(x_train, y_train, 'ro', markersize=6, label="training points") # noiseless data
#ax1.plot(x_test, y_test, 'kX', markersize=6, label="testing points") # Plot test points

ax1.set_xlabel('$t (s)$', fontsize=20)
ax1.set_ylabel('$r (m)$', fontsize=20)
ax1.set_title("Posterior kernel: %s"
              % gp_model.kernel_, fontsize=20) # Show in the title the value of the hyperparameters
#ax1.set_ylim(-10, 15) # just to provide more space for the legend
ax1.legend(loc='upper left', fontsize=15)
fig1.set_size_inches(8,8)
plt.close(fig1) # close the plot to see it in next cell

In [ ]:
fig1

<g id='Training with Different Kernels: Matern with no optimizer'></g>

### Training with Different Kernels: Matern with no optimizer

In [ ]:
# Define the kernel function
kernel = Matern(length_scale=1.0, length_scale_bounds=(1e-2, 1e2),nu=1.5)

gp_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, optimizer=None, n_restarts_optimizer=20)

# Fit to data to determine parameters
gp_model.fit(X_train, y_train)

# Make the prediction on the entire dataset (for plotting)
y_plot_pred, sigma_plot = gp_model.predict(X_plot, return_std=True) # also output the uncertainty (std)

# Predict for test set (for error metric)
y_pred, sigma = gp_model.predict(X_test, return_std=True) # also output the uncertainty (std)

In [ ]:
# Plot the prediction and the 95% confidence interval
fig1, ax1 = plt.subplots()

ax1.plot(x_plot, y_plot_pred, 'b-', label="GPR prediction")
ax1.fill(np.concatenate([x_plot, x_plot[::-1]]),
         np.concatenate([y_plot_pred - 1.9600 * sigma_plot,
                        (y_plot_pred + 1.9600 * sigma_plot)[::-1]]),
         alpha=.5, fc='b', ec='None', label='95% confidence interval')

#ax1.plot(x_train, y_train, 'ro', markersize=6, label="training points") # noiseless data
#ax1.plot(x_test, y_test, 'kX', markersize=6, label="testing points") # Plot test points

ax1.set_xlabel('$t (s)$', fontsize=20)
ax1.set_ylabel('$r (m)$', fontsize=20)
ax1.set_title("Posterior kernel: %s"
              % gp_model.kernel_, fontsize=20) # Show in the title the value of the hyperparameters
#ax1.set_ylim(-10, 15) # just to provide more space for the legend
ax1.legend(loc='upper left', fontsize=15)
fig1.set_size_inches(8,8)
plt.close(fig1) # close the plot to see it in next cell

In [ ]:
fig1

### different optimizers? different ratios of training/testing data?

<h id='Phase 1.5: Gaussian Process on Experimental Data Only'></h>

# Phase 1.5: Gaussian Process on Experimental Data Only

In [ ]:
exp_df = pd.read_csv("PolyAcry_12_28_04") # read pandas DataFrame
print("Show the experimental DataFrame used for testing data:\n", exp_df)

In [ ]:
Data_t_exp = exp_df.iloc[:,0]
Data_R1_exp = exp_df.iloc[:,1] 
Data_R2_exp = exp_df.iloc[:,2] 
print("Experimental x data is time:\n",Data_t_exp)
print("Experimental Bubble Radius from camera one is:\n",Data_R1_exp)
print("Experimental Bubble Radius from camera two is:\n",Data_R2_exp)

In [ ]:
fig_exp_data, ax_exp_data = plt.subplots() # create a plot
R1_line = ax_exp_data.plot(Data_t_exp, Data_R1_exp, 'b.', label='R1 of camera one at 0 degrees')
R2_line = ax_exp_data.plot(Data_t_exp, Data_R2_exp, 'r.',label='R2 of camera two at 90 degrees')
ax_exp_data.set_xlabel("t (s)", fontsize=20) # create x-axis label with font size 20
ax_exp_data.set_ylabel("r (m)", fontsize=20) # create y-axis label with font size 20
ax_exp_data.set_title("Experimental Bubble Radius vs Time", fontsize=20); # create title with font size 20

handles, labels = ax_exp_data.get_legend_handles_labels()
ax_exp_data.legend(handles, labels)
plt.show(fig_exp_data)

In [ ]:
n_data = len(Data_t_exp) # number of points in our dataset
testset_ratio = 0.75 # ratio of test set points from the dataset

t_data = np.reshape(Data_t_exp,(-1,1)) # a 2D array that scikit-learn likes

seed = 1987 # set a random seed so that everyone gets the same result
np.random.seed(seed)

# Let's split into 10% training points and the rest for testing:
X_train, X_test, y_train, y_test = train_test_split(t_data,
                                    Data_R1_exp, test_size=testset_ratio,
                                    random_state=seed)

x_train = X_train.ravel() # just for plotting later
x_test = X_test.ravel() # just for plotting later

print("Here's a print of X_train:\n", X_train)

In [ ]:
n_exp_max = np.max(t_data)
print(n_exp_max)

# Define points used for plotting
n_plot = 500 # number of points used for plotting the model
n_max = np.max(t_data)
x_exp_plot = np.linspace(0, n_exp_max, n_plot) # create dataset with uniformly spaced points
X_exp_plot = np.reshape(x_exp_plot,(-1,1)) # a 2D array that scikit-learn likes

<i id='Training with Different kernels: common Constant*RBF kernel with an optimizer fmin_1_bfgs_b'></i>

## Training with Different kernels: common Constant*RBF kernel with an optimizer fmin_1_bfgs_b

In [ ]:
# Define the kernel function
kernel = ConstantKernel(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2)) # This is the very common Constant*RBF kernel

gp_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-2, optimizer='fmin_l_bfgs_b', n_restarts_optimizer=20)
#gp_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-10, optimizer=None, n_restarts_optimizer=20)

# Fit to data to determine parameters
gp_model.fit(X_train, y_train)

# Make the prediction on the entire dataset (for plotting)
y_plot_pred, sigma_plot = gp_model.predict(X_exp_plot, return_std=True) # also output the uncertainty (std)

# Predict for test set (for error metric)
y_pred, sigma = gp_model.predict(X_test, return_std=True) # also output the uncertainty (std)

In [ ]:
# Plot the prediction and the 95% confidence interval
fig1, ax1 = plt.subplots()

ax1.plot(x_exp_plot, y_plot_pred, 'b-', label="GPR prediction")
ax1.fill(np.concatenate([x_exp_plot, x_exp_plot[::-1]]),
         np.concatenate([y_plot_pred - 1.9600 * sigma_plot,
                        (y_plot_pred + 1.9600 * sigma_plot)[::-1]]),
         alpha=.5, fc='b', ec='None', label='95% confidence interval')

ax1.plot(x_train, y_train, 'ro', markersize=6, label="training points") # noiseless data
ax1.plot(x_test, y_test, 'kX', markersize=6, label="testing points") # Plot test points

ax1.set_xlabel('$t (s)$', fontsize=20)
ax1.set_ylabel('$r (m)$', fontsize=20)
ax1.set_title("Posterior kernel: %s"
              % gp_model.kernel_, fontsize=20) # Show in the title the value of the hyperparameters
#ax1.set_ylim(-10, 15) # just to provide more space for the legend
ax1.legend(loc='upper left', fontsize=15)
fig1.set_size_inches(8,8)
plt.close(fig1) # close the plot to see it in next cell

In [ ]:
fig1

<j id='Training with Different Kernels: ExpSineSquared with an optimizer fmin_1_bfgs_b'></j>

### Training with Different Kernels: ExpSineSquared with an optimizer fmin_1_bfgs_b

In [ ]:
# Define the kernel function
kernel = ExpSineSquared(length_scale=3.0, periodicity=3.14,
                        length_scale_bounds=(0.1, 10.0),
                        periodicity_bounds=(0.1, 10)) * RBF(3.0, (1e-2, 1e2))

gp_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, optimizer='fmin_l_bfgs_b', n_restarts_optimizer=20)
#gp_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-10, optimizer=None, n_restarts_optimizer=20)

# Fit to data to determine parameters
gp_model.fit(X_train, y_train)

# Make the prediction on the entire dataset (for plotting)
y_plot_pred, sigma_plot = gp_model.predict(X_exp_plot, return_std=True) # also output the uncertainty (std)

# Predict for test set (for error metric)
y_pred, sigma = gp_model.predict(X_test, return_std=True) # also output the uncertainty (std)

In [ ]:
# Plot the prediction and the 95% confidence interval
fig1, ax1 = plt.subplots()

ax1.plot(x_exp_plot, y_plot_pred, 'b-', label="GPR prediction")
ax1.fill(np.concatenate([x_exp_plot, x_exp_plot[::-1]]),
         np.concatenate([y_plot_pred - 1.9600 * sigma_plot,
                        (y_plot_pred + 1.9600 * sigma_plot)[::-1]]),
         alpha=.5, fc='b', ec='None', label='95% confidence interval')

ax1.plot(x_train, y_train, 'ro', markersize=6, label="training points") # noiseless data
ax1.plot(x_test, y_test, 'kX', markersize=6, label="testing points") # Plot test points

ax1.set_xlabel('$t (s)$', fontsize=20)
ax1.set_ylabel('$r (m)$', fontsize=20)
ax1.set_title("Posterior kernel: %s"
              % gp_model.kernel_, fontsize=20) # Show in the title the value of the hyperparameters
#ax1.set_ylim(-10, 15) # just to provide more space for the legend
ax1.legend(loc='upper left', fontsize=15)
fig1.set_size_inches(8,8)
plt.close(fig1) # close the plot to see it in next cell

In [ ]:
fig1

<k id='Training with Different Kernels: Matern with an optimizer fmin_1_bfgs_b'></k>

### Training with Different Kernels: Matern with an optimizer fmin_1_bfgs_b

In [ ]:
# Define the kernel function
kernel = Matern(length_scale=1.0, length_scale_bounds=(1e-2, 1e2),nu=1.5)

gp_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, optimizer='fmin_l_bfgs_b', n_restarts_optimizer=20)
#gp_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, optimizer=None, n_restarts_optimizer=20)

# Fit to data to determine parameters
gp_model.fit(X_train, y_train)

# Make the prediction on the entire dataset (for plotting)
y_plot_pred, sigma_plot = gp_model.predict(X_exp_plot, return_std=True) # also output the uncertainty (std)

# Predict for test set (for error metric)
y_pred, sigma = gp_model.predict(X_test, return_std=True) # also output the uncertainty (std)

In [ ]:
# Plot the prediction and the 95% confidence interval
fig1, ax1 = plt.subplots()

ax1.plot(x_exp_plot, y_plot_pred, 'b-', label="GPR prediction")
ax1.fill(np.concatenate([x_exp_plot, x_exp_plot[::-1]]),
         np.concatenate([y_plot_pred - 1.9600 * sigma_plot,
                        (y_plot_pred + 1.9600 * sigma_plot)[::-1]]),
         alpha=.5, fc='b', ec='None', label='95% confidence interval')

ax1.plot(x_train, y_train, 'ro', markersize=6, label="training points") # noiseless data
ax1.plot(x_test, y_test, 'kX', markersize=6, label="testing points") # Plot test points

ax1.set_xlabel('$t (s)$', fontsize=20)
ax1.set_ylabel('$r (m)$', fontsize=20)
ax1.set_title("Posterior kernel: %s"
              % gp_model.kernel_, fontsize=20) # Show in the title the value of the hyperparameters
#ax1.set_ylim(-10, 15) # just to provide more space for the legend
ax1.legend(loc='upper left', fontsize=15)
fig1.set_size_inches(8,8)
plt.close(fig1) # close the plot to see it in next cell

In [ ]:
fig1

## with no optimizer? or different optimizers?

<l id='Phase 2: Training with Simulation Data and Testing with Experimental Data'></l>

# Phase 2: Training with Simulation Data and Testing with Experimental Data
## running with Constant Kernel and  f_min optimizer, Matern kernel with no optimizer

In [ ]:
# training data = simulation data, test data = experimental data

# defining training data
X_train = X_data # a 2D array that scikit-learn likes, for simulated data
y_train = Data_y

# defining test data
X_test = t_data
y_test = Data_R1_exp
#y_train2 = Data_R2_exp

n_data = len(Data_t_exp) # number of points in our dataset
#testset_ratio = len(X_train)/len(Data_t_exp) # ratio of test set points from the datasets

x_train = X_train.ravel() # just for plotting later
x_test = X_test.ravel() # just for plotting later

print(len(X_train))
print(len(y_train))

print(len(X_test))
print(len(y_test))

<m id='Constant Kernel and  f_min optimizer'></m>

### Constant Kernel and  f_min optimizer

In [ ]:
# Define the kernel function
kernel = ConstantKernel(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2)) # This is the very common Constant*RBF kernel

gp_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, optimizer='fmin_l_bfgs_b', n_restarts_optimizer=20)

# Fit to data to determine parameters
gp_model.fit(X_train, y_train)

# Make the prediction on the entire dataset (for plotting)
y_plot_pred, sigma_plot = gp_model.predict(X_plot, return_std=True) # also output the uncertainty (std)

# Predict for test set (for error metric)
y_pred, sigma = gp_model.predict(X_test, return_std=True) # also output the uncertainty (std)

In [ ]:
# Plot the prediction and the 95% confidence interval
fig1, ax1 = plt.subplots()

ax1.plot(x_exp_plot, y_plot_pred, 'b-', label="GPR prediction")
ax1.fill(np.concatenate([x_exp_plot, x_exp_plot[::-1]]),
         np.concatenate([y_plot_pred - 1.9600 * sigma_plot,
                        (y_plot_pred + 1.9600 * sigma_plot)[::-1]]),
         alpha=.5, fc='b', ec='None', label='95% confidence interval')

#ax1.plot(x_train, y_train, 'ro', markersize=6, label="training points") # noiseless data
#ax1.plot(x_test, y_test, 'kX', markersize=6, label="testing points") # Plot test points

ax1.set_xlabel('$t (s)$', fontsize=20)
ax1.set_ylabel('$r (m)$', fontsize=20)
ax1.set_title("Posterior kernel: %s"
              % gp_model.kernel_, fontsize=20) # Show in the title the value of the hyperparameters
#ax1.set_ylim(-10, 15) # just to provide more space for the legend
ax1.legend(loc='upper left', fontsize=15)
fig1.set_size_inches(8,8)
plt.close(fig1) # close the plot to see it in next cell

In [ ]:
fig1

<n id='Matern kernel with no optimizer'></n>

### Matern kernel with no optimizer

In [ ]:
# Define the kernel function
kernel = Matern(length_scale=1.0, length_scale_bounds=(1e-2, 1e2),nu=1.5)

gp_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, optimizer=None, n_restarts_optimizer=20)

# Fit to data to determine parameters
gp_model.fit(X_train, y_train)

# Make the prediction on the entire dataset (for plotting)
y_plot_pred, sigma_plot = gp_model.predict(X_plot, return_std=True) # also output the uncertainty (std)

# Predict for test set (for error metric)
y_pred, sigma = gp_model.predict(X_test, return_std=True) # also output the uncertainty (std)

In [ ]:
# Plot the prediction and the 95% confidence interval
fig1, ax1 = plt.subplots()

ax1.plot(x_exp_plot, y_plot_pred, 'b-', label="GPR prediction")
ax1.fill(np.concatenate([x_exp_plot, x_exp_plot[::-1]]),
         np.concatenate([y_plot_pred - 1.9600 * sigma_plot,
                        (y_plot_pred + 1.9600 * sigma_plot)[::-1]]),
         alpha=.5, fc='b', ec='None', label='95% confidence interval')

#ax1.plot(x_train, y_train, 'ro', markersize=6, label="training points") # noiseless data
#ax1.plot(x_test, y_test, 'kX', markersize=6, label="testing points") # Plot test points

ax1.set_xlabel('$t (s)$', fontsize=20)
ax1.set_ylabel('$r (m)$', fontsize=20)
ax1.set_title("Posterior kernel: %s"
              % gp_model.kernel_, fontsize=20) # Show in the title the value of the hyperparameters
#ax1.set_ylim(-10, 15) # just to provide more space for the legend
ax1.legend(loc='upper left', fontsize=15)
fig1.set_size_inches(8,8)
plt.close(fig1) # close the plot to see it in next cell

In [ ]:
fig1

<o id='Phase 3: Multi-fidelity Gaussian Process'></o>

# Phase 3: Multi-fidelity Gaussian Process

In [ ]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# Setting up Training and Testing Data
#Training data is 100 points in [0,1] inclusive regularly spaced
train_x = torch.linspace(0, 1, 100)
# True function is sin(2*pi*x) with Gaussian noise
train_y = torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * math.sqrt(0.04)

In [ ]:
#botorch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood

#fit model
train_X = torch.rand(10, 2)
Y = 1 - torch.linalg.norm(train_X - 0.5, dim=-1, keepdim=True)
Y = Y + 0.1 * torch.randn_like(Y)  # add some noise
train_Y = standardize(Y)

gp = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll)

In [ ]:
#construct acquisition function
from botorch.acquisition import UpperConfidenceBound

UCB = UpperConfidenceBound(gp, beta=0.1)

In [ ]:
#optimize acquisition function
from botorch.optim import optimize_acqf

bounds = torch.stack([torch.zeros(2), torch.ones(2)])
candidate, acq_value = optimize_acqf(
    UCB, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
)
candidate  # tensor([0.4887, 0.5063])

In [ ]:
#gpytorch
#setting up training/testing data
#setting up the model
#training the model
#make predictions with the model
#plot the model fit 

<p id='Phase 4: Comparing Error Metrics'></p>

# Phase 4: Comparing Error Metrics

### # Phase 5: Different Materials? More data files? Different shear moduli/viscosity (hyperparameters?)
## Slides (12) for presentation: 2 GP (M), 4 data-assim (V), 4 - multi-fidelity (M), 2 - ? (V)
### want understanding: why chose those algorithms, compare algorithms, explain new methods used

### HAVE TO SCALE DATA